<a href="https://colab.research.google.com/github/boomshineking/mlproject/blob/main/quatized_8B_3_1Llama_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

Model page: https://huggingface.co/Richard9905/quatized-8B-3.1Llama-model

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Richard9905/quatized-8B-3.1Llama-model)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Richard9905/quatized-8B-3.1Llama-model")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Richard9905/quatized-8B-3.1Llama-model")
model = AutoModelForCausalLM.from_pretrained("Richard9905/quatized-8B-3.1Llama-model")

In [1]:
pip install transformers accelerate datasets peft bitsandbytes trl


In [6]:
from huggingface_hub import login

login()


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import Trainer, DataCollatorForLanguageModeling

# Load quantized model and tokenizer
model_name = "Richard9905/quatized-8B-3.1Llama-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")

# Load dataset
dataset = load_dataset("Richard9905/finetuned_bibleKJV_Dataset")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


train.json:   0%|          | 0.00/9.71M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/1.09M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1365 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/152 [00:00<?, ? examples/s]

In [12]:
def format_prompt(example):
    return {
        "input_ids": tokenizer(
            f"### Instruction:\n{example['user']}\n\n### Response:\n{example['assistant']}",
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )["input_ids"][0]
    }
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
training_args = TrainingArguments(
    output_dir="./lora-bible-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    save_steps=100,
    report_to="none"
)
training_args = TrainingArguments(
    output_dir="./lora-bible-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    save_steps=100,
    report_to="none"
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()



/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


NameError: name 'tokenized_dataset' is not defined

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

# Load quantized model and tokenizer
model_name = "Richard9905/quatized-8B-3.1Llama-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")

# Load dataset
dataset = load_dataset("Richard9905/finetuned_bibleKJV_Dataset")

# Format function
def format_prompt(example):
    prompt = f"### Instruction:\n{example['user']}\n\n### Response:\n{example['assistant']}"
    tokens = tokenizer(prompt, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    return {"input_ids": tokens["input_ids"][0], "attention_mask": tokens["attention_mask"][0]}

# Apply formatting
tokenized_dataset = dataset.map(format_prompt)

# Setup LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Only apply LoRA once
model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./lora-bible-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    save_steps=100,
    report_to="none"
)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/1365 [00:00<?, ? examples/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [14]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

# Load model and tokenizer with 4-bit quantization using BitsAndBytesConfig
model_name = "Richard9905/quatized-8B-3.1Llama-model"

bnb_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Required for padding

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load dataset
dataset = load_dataset("Richard9905/finetuned_bibleKJV_Dataset")

# Split if needed
if "train" not in dataset or "test" not in dataset:
    dataset = dataset["train"].train_test_split(test_size=0.1)

# Format prompt for instruction-tuning
def format_prompt(example):
    prompt = f"### Instruction:\n{example['user']}\n\n### Response:\n{example['assistant']}"
    tokens = tokenizer(
        prompt,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    return {
        "input_ids": tokens["input_ids"][0],
        "attention_mask": tokens["attention_mask"][0],
        "labels": tokens["input_ids"][0]
    }

# Apply formatting
tokenized_dataset = dataset.map(format_prompt)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply PEFT (LoRA)
model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./lora-bible-model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    save_steps=100,
    report_to="none"
)

# Data collator (for CausalLM)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Repo card metadata block was not found. Setting CardData to empty.


Map:   0%|          | 0/1365 [00:00<?, ? examples/s]

Map:   0%|          | 0/152 [00:00<?, ? examples/s]

/tmp/ipython-input-14-1652490861.py:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.709500
20,0.557000
30,0.380700
40,0.337600
50,0.318000
60,0.312800
70,0.290700
80,0.282400
90,0.284000
100,0.266500


TrainOutput(global_step=513, training_loss=0.2532037302532159, metrics={'train_runtime': 801.3704, 'train_samples_per_second': 5.11, 'train_steps_per_second': 0.64, 'total_flos': 9.445362207031296e+16, 'train_loss': 0.2532037302532159, 'epoch': 3.0})

In [15]:
model.push_to_hub("Richard9905/lora-bible-model", use_temp_dir=True)
tokenizer.push_to_hub("Richard9905/lora-bible-model", use_temp_dir=True)


adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Richard9905/lora-bible-model/commit/bef6fbdec203c969474a6a296544b4abf3dfa2cb', commit_message='Upload tokenizer', commit_description='', oid='bef6fbdec203c969474a6a296544b4abf3dfa2cb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Richard9905/lora-bible-model', endpoint='https://huggingface.co', repo_type='model', repo_id='Richard9905/lora-bible-model'), pr_revision=None, pr_num=None)

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from huggingface_hub import login



# ---- 2. Load Quantized Base Model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Or load_in_8bit=True
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

base_model = AutoModelForCausalLM.from_pretrained(
    "Richard9905/quatized-8B-3.1Llama-model",
    quantization_config=bnb_config,
    device_map="auto"
)

# ---- 3. Load LoRA Adapter
model = PeftModel.from_pretrained(base_model, "Richard9905/lora-bible-model")

# ---- 4. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("Richard9905/lora-bible-model")

# ---- 5. Merge LoRA weights into base model
merged_model = model.merge_and_unload()

# ---- 6. Save Merged Model Locally
save_directory = "merged-bible-model"
merged_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# ---- 7. Push to Hugging Face
merged_model.push_to_hub("Richard9905/full-merged-bible-model")
tokenizer.push_to_hub("Richard9905/full-merged-bible-model")


/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/793 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Richard9905/full-merged-bible-model/commit/0178718ec1505cae3ae0d9e9c9a541f471302d5e', commit_message='Upload tokenizer', commit_description='', oid='0178718ec1505cae3ae0d9e9c9a541f471302d5e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Richard9905/full-merged-bible-model', endpoint='https://huggingface.co', repo_type='model', repo_id='Richard9905/full-merged-bible-model'), pr_revision=None, pr_num=None)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "Richard9905/full-merged-bible-model"

# Load tokenizer and model (Colab should have enough VRAM if you're using a Pro plan)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

# Build generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Your prompt
prompt = "### Instruction:\nWho is Jesus?\n\n### Response:\n"

# Generate text
response = pipe(prompt, max_new_tokens=150, temperature=0.7)[0]["generated_text"]

print(response)


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


### Instruction:
Who is Jesus?

### Response:
Jesus is the son of God and the savior of humanity. He is considered one of the most important figures in Christianity, and his teachings, life, death, and resurrection are the central events of the Christian faith.

### Analysis:
This question requires the ability to understand and articulate the fundamental beliefs of Christianity. To answer this question, one must be able to comprehend the role of Jesus in the Christian faith and express it in a clear and concise manner.

### Explanation:
Jesus is the central figure of Christianity, and his significance extends beyond his role as a historical figure. He is seen as the son of God, the savior of humanity, and the embodiment of God's love and redemption. His teachings, which include the importance of love, forgiveness
